In [ ]:
import sys
import numpy as np
from tqdm import tqdm_notebook
import pickle
import pandas as pd
import plotly
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import time
import init as x
import simulate_corona as sc
import simulate_corona_senior as scs
import multiprocessing as mp
sns.set(rc={'figure.figsize':(11, 4)})

## initialize
np.random.seed(123)
percSIR_high=[]
percSIR_med=[]
percSIR_low=[]
percSIR_low_senior=[]
SIR_high=[]
SIR_med=[]
SIR_low=[]
SIR_low_senior=[]

cores_cnt = mp.cpu_count()
# pool = mp.Pool(cores_cnt)

alpha_vec_sen = np.full(x.simul_cnt, x.public_trans_low)
alpha_vec_low = np.full(x.simul_cnt, x.public_trans_low)
alpha_vec_mid = np.full(x.simul_cnt, x.public_trans_mid)
alpha_vec_high = np.full(x.simul_cnt, x.public_trans_high)
run_multicore = True

def run_simulation(alpha_vec,fun):
    ra = []
    rb = []
    if run_multicore:
        pool = mp.Pool(cores_cnt)
        tic = time.perf_counter()
        res = pool.map(fun,alpha_vec)
        toc = time.perf_counter()
        pool.close()
        dt_avg = (toc-tic)/x.simul_cnt
        k=0
        for sim in np.arange(x.simul_cnt):
            ra0,rb0 = res[k]
            ra.append(ra0)
            rb.append(rb0)
            k +=1
    else:
        k=0
        tic = time.perf_counter()
        for sim in np.arange(x.simul_cnt):    
            print(k)
            ra0,rb0 = sc.simul(alpha_vec[k])
            ra.append(ra0)
            ra.append(rb0)
            k +=1
        toc = time.perf_counter()
        dt_avg = (toc-tic)/x.simul_cnt
    return ra,rb,dt_avg      
    

## run all simulations
percSIR_high,SIR_high,dt_avg_high=run_simulation(alpha_vec_high,sc.simul)
print(dt_avg_high)
percSIR_med,SIR_med,dt_avg_med=run_simulation(alpha_vec_mid,sc.simul)
print(dt_avg_med)
#percSIR_low,SIR_low=run_simulation(alpha_vec_low,sc.simul)
#percSIR_low_senior,SIR_low_senior=run_simulation(alpha_vec_sen,scs.simul)
